In [94]:
import os
import json
import pygments
from gensim.models import Word2Vec 
import numpy as np
from scipy import spatial

In [42]:
with open('./index/file_key.txt', 'r') as f:
    id_to_f = json.loads(f.read())['id-to-f']
with open('./temp.json', 'r') as f:
    function_defs = json.loads(f.read())

In [53]:

class NameFormatter(pygments.formatter.Formatter):
    def __init__(self):
        self.names = []

    def format(self, tokensource, outfile):
        for _type, value in tokensource:
            if _type[0] == "Name" or _type[0] == "Keyword":
                self.names.append(value)
    
    def get_tok_arr(self):
        return self.names

In [117]:
f_to_toklist = {}
sentences = []
for f in function_defs:
    with open(os.path.join('./raw', f.replace('.json', '.py'))) as fr:
        line_arr = fr.readlines()
        for fd in function_defs[f]:
            for s, e in zip(function_defs[f][fd]['lineno'], function_defs[f][fd]['line-end']):
                func_body = ''.join(line_arr[s:e+2])                   
                formatter = NameFormatter()                               
                pygments.highlight(func_body, pygments.lexers.PythonLexer(), formatter)
                f_to_toklist[f"{f}|{fd}"] = formatter.get_tok_arr()
                sentences.append(formatter.get_tok_arr())

['login', 'gh', 'get_user', 'login', 'issues', 'gh', 'search_issues', 'args', 'issue_repo', 'title', 'login', 'first_page', 'issues', 'get_page', 'return', 'first_page', 'if', 'first_page', 'else', 'None']
['link', 'args', 'ci_name', 'args', 'link_to_ci_run', 'issue', 'get_issue', 'max_body_length', 'original_body_length', 'len', 'body', 'if', 'original_body_length', 'max_body_length', 'body', 'body', 'max_body_length', 'original_body_length', 'if', 'issue', 'None', 'header', 'link', 'date_str', 'issue', 'issue_repo', 'create_issue', 'title', 'title', 'body', 'header', 'body', 'print', 'args', 'issue_repo', 'issue', 'number', 'sys', 'exit', 'else', 'header', 'link', 'date_str', 'issue', 'edit', 'body', 'header', 'body', 'print', 'args', 'issue_repo', 'issue', 'number', 'sys', 'exit']
['print', 'issue', 'get_issue', 'if', 'issue', 'None', 'header_str', 'comment_str', 'header_str', 'args', 'link_to_ci_run', 'date_str', 'print', 'issue', 'number', 'for', 'comment', 'issue', 'get_comments'

KeyboardInterrupt: 

In [62]:
model = Word2Vec(window = 4, min_count = 3, vector_size = 30)
model.build_vocab(corpus_iterable=sentences)
model.train(corpus_iterable=sentences, total_examples=len(sentences), epochs=15)

(4837122, 6435855)

In [92]:
N = len(sentences)
vec_length = model.wv['float'].shape[0]
vec_mat = np.zeros((N, vec_length))
id_to_f = []
f_to_id = {}
for f in function_defs:
    for fd in function_defs[f]:
        fqname = f"{f}|{fd}"
        f_vec = np.mean(np.asarray([model.wv[t] if t in model.wv else np.zeros(vec_length) for t in f_to_toklist[fqname]]), axis=0)
        vec_mat[len(id_to_f)] = f_vec
        f_to_id[fqname] = len(id_to_f)
        id_to_f.append(fqname)
        

/home/chirag/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/chirag/.local/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [97]:
tree = spatial.KDTree(vec_mat)

In [116]:
lookup = vec_mat[f_to_id['scikit-learn/sklearn/mixture/_base.json|_estimate_weighted_log_prob']]
print(f_to_id['scikit-learn/sklearn/mixture/_base.json|_estimate_weighted_log_prob'])
print(lookup)
[id_to_f[id] for id in tree.query(lookup, k=6)[1][1:]]



3673
[-1.89802204  1.10554934 -0.85192186 -0.88560346  0.32426918 -0.54258606
 -0.405833    0.8672047   0.40095198  0.2243812   0.06561348  0.17665482
 -2.08564311  0.00328436  0.1254919  -0.81854913 -0.01213146  0.58601207
  0.44412011  1.2289026   0.34645278  0.35993355 -0.74876561 -1.60863462
  0.94687962  0.18033799  0.22918296  0.37969161 -0.61898294  1.55141755]


['scikit-learn/sklearn/mixture/_bayesian_mixture.json|_check_parameters',
 'scikit-learn/sklearn/impute/tests/test_base.json|transform',
 'scikit-learn/sklearn/svm/_base.json|_decision_function',
 'scikit-learn/sklearn/tests/test_metaestimators.json|inverse_transform',
 'scikit-learn/sklearn/tests/test_metaestimators.json|transform']